In [9]:
import os
import json

#load data
file_name = "./data/actual.json"

with open(file_name, 'r') as file:
    data = json.load(file) 


In [10]:
#find vector dimensions for occurrence of city and merchandise in one route so all 
#cities and merchandise characterize a route
vector=set()
for route in data:
    for trip in route["route"]:
        vector.update([trip["from"],trip["to"]])
        for merch in trip["merchandise"]:
            vector.update([merch])
print(list(vector)[:10])
print(len(vector))  

['Yogurt', 'Udine', 'Firenze', 'Pisa', 'Tomatoes', 'Meat', 'Apples', 'Verona', 'Venezia', 'Milano']
40


In [11]:
#find vector dimensions for combination of cities (so a connection between cities)
#and occurrence of a specific merchandise in the entire route
vector=set()
for route in data:
    for trip in route["route"]:
        vector.update([trip["from"]+"-"+trip["to"]])
        for merch in trip["merchandise"]:
            vector.update([merch])
print(list(vector)[:10])
print(len(vector))  

['Rome-Palermo', 'Udine-Palermo', 'Padua-Pisa', 'Siena-Modena', 'Modena-Lecce', 'Firenze-Pisa', 'Bolzano-Monza', 'Pisa-Milano', 'Napoli-Trento', 'Palermo-Modena']
400


In [12]:
#find vector dimensions for combination of cities (so a connection between cities) in
#combination with the merchandise that is carried on that route
vector=set()
for route in data:
    for trip in route["route"]:
        for merch in trip["merchandise"]:
            vector.update([trip["from"]+"-"+trip["to"]+"-"+merch])
print(list(vector)[:10])
print(len(vector))  

['Trento-Verona-Tomatoes', 'Siena-Lecce-Rice', 'Verona-Rome-Milk', 'Bologna-Palermo-Butter', 'Rome-Napoli-Rice', 'Modena-Verona-Pasta', 'Bergamo-Udine-Tea', 'Genova-Napoli-Milk', 'Napoli-Modena-Rice', 'Padua-Udine-Honey']
7351


In [13]:
#find vector dimensions for combination of cities (so a connection between cities) in
#combination with the merchandise and the amount that is carried on that route
vector=set()
for route in data:
    for trip in route["route"]:
        for merch in trip["merchandise"]:
            vector.update([trip["from"]+"-"+trip["to"]+"-"+merch+"-"+str(trip["merchandise"][merch])])
print(list(vector)[:10])
print(len(vector))

['Genova-Rome-Pasta-6', 'Genova-Firenze-Potatoes-9', 'Torino-Udine-Water-30', 'Udine-Trento-Butter-8', 'Torino-Genova-Tomatoes-29', 'Torino-Rome-Meat-5', 'Siena-Udine-Chocolate-17', 'Siena-Trento-Bananas-14', 'Padua-Milano-Potatoes-3', 'Rome-Padua-Yogurt-29']
77152


In [16]:
#find vector dimensions for combination of cities (so a connection between cities) in
#combination with the merchandise and the amount that is carried on that route where 
#the amount is specified by a indicator (small, medium, large)
#to do that the max and min is found for every merchandise
mapminmax={}
possiblecomb={}
vector=set()

for route in data:
    for trip in route["route"]:
        for merch in trip["merchandise"]:
            if trip["from"]+"-"+trip["to"]+"-"+merch+"-min" in mapminmax:
                mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-min"]=min(mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-min"], trip["merchandise"][merch])
            else:
                mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-min"]= trip["merchandise"][merch]
            if trip["from"]+"-"+trip["to"]+"-"+merch+"-max" in mapminmax:
                mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-max"]=max(mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-max"], trip["merchandise"][merch])
            else:
                mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-max"]= trip["merchandise"][merch]
            
            if trip["from"]+"-"+trip["to"]+"-"+merch in possiblecomb:
                possiblecomb[trip["from"]+"-"+trip["to"]+"-"+merch]=possiblecomb[trip["from"]+"-"+trip["to"]+"-"+merch]+1
            else:
                possiblecomb[trip["from"]+"-"+trip["to"]+"-"+merch]=1

mapborders={}
for item in possiblecomb:
    if possiblecomb[item]>1:
        smallmediumborder=mapminmax[item+"-"+"min"]+((1/3)*(mapminmax[item+"-"+"max"]-mapminmax[item+"-"+"min"]))
        mediumlargeborder=mapminmax[item+"-"+"min"]+((2/3)*(mapminmax[item+"-"+"max"]-mapminmax[item+"-"+"min"]))
        mapborders[item]=[smallmediumborder, mediumlargeborder]
    else:
        mapborders[item]=[0]


for route in data:
    for trip in route["route"]:
        for merch in trip["merchandise"]:
            if mapborders[trip["from"]+"-"+trip["to"]+"-"+merch][0]==0:
                vector.update([trip["from"]+"-"+trip["to"]+"-"+merch])
            else:
                if trip["merchandise"][merch]<mapborders[trip["from"]+"-"+trip["to"]+"-"+merch][0]:
                    vector.update([trip["from"]+"-"+trip["to"]+"-"+merch+"-"+"small"])
                elif trip["merchandise"][merch]>mapborders[trip["from"]+"-"+trip["to"]+"-"+merch][1]:
                    vector.update([trip["from"]+"-"+trip["to"]+"-"+merch+"-"+"large"])
                else:
                    vector.update([trip["from"]+"-"+trip["to"]+"-"+merch+"-"+"medium"])

print(possiblecomb)
print(mapborders)
print(list(vector)[:20])
print(len(vector))

{'Milano-Genova-Tea': 4298, 'Milano-Genova-Meat': 4298, 'Milano-Genova-Pasta': 4295, 'Milano-Genova-Water': 4054, 'Genova-Siena-Pasta': 4370, 'Genova-Siena-Pens': 4367, 'Genova-Siena-Meat': 4376, 'Genova-Siena-Honey': 8881, 'Genova-Siena-Butter': 8890, 'Genova-Siena-Bread': 4133, 'Siena-Trento-Carrots': 4467, 'Siena-Trento-Honey': 4481, 'Siena-Trento-Tomatoes': 4229, 'Siena-Trento-Rice': 4506, 'Siena-Trento-Apples': 4482, 'Trento-Lecce-Pasta': 8033, 'Trento-Lecce-Fish': 3867, 'Trento-Lecce-Apples': 4093, 'Trento-Lecce-Milk': 3776, 'Lecce-Bologna-Water': 3564, 'Lecce-Bologna-Tea': 3811, 'Lecce-Bologna-Apples': 3811, 'Lecce-Bologna-Potatoes': 3819, 'Lecce-Bologna-Bananas': 3702, 'Lecce-Bologna-Tomatoes': 3683, 'Bologna-Bolzano-Potatoes': 4393, 'Bologna-Bolzano-Cheese': 4403, 'Bologna-Bolzano-Beer': 4411, 'Bologna-Bolzano-Chocolate': 4388, 'Bologna-Bolzano-Water': 4149, 'Bolzano-Rome-Bread': 82, 'Bolzano-Rome-Fish': 81, 'Bolzano-Rome-Butter': 4702, 'Rome-Padua-Pasta': 98, 'Rome-Padua-Meat